### PARTE II

### Análise Estatística Preliminar

#### Dataset:

Pesquisa Nacional por Amostra de Domicílios - 2015

A <b>Pesquisa Nacional por Amostra de Domicílios - PNAD</b> investiga anualmente, de forma permanente, características gerais da população, de educação, trabalho, rendimento e habitação e outras, com periodicidade variável, de acordo com as necessidades de informação para o país, como as características sobre migração, fecundidade, nupcialidade, saúde, segurança alimentar, entre outros temas. O levantamento dessas estatísticas constitui, ao longo dos 49 anos de realização da pesquisa, um importante instrumento para formulação, validação e avaliação de políticas orientadas para o desenvolvimento socioeconômico e a melhoria das condições de vida no Brasil.

Fonte dos dados: https://ww2.ibge.gov.br/home/estatistica/populacao/trabalhoerendimento/pnad2015/microdados.shtm

### Variáveis utilizadas

> ### Renda
> ***

Rendimento mensal do trabalho principal para pessoas de 10 anos ou mais de idade.

> ### Idade
> ***

Idade do morador na data de referência em anos.

> ### Altura (elaboração própria)
> ***

Altura do morador em metros.

> ### UF
> ***

|Código|Descrição|
|---|---|
|11|Rondônia|
|12|Acre|
|13|Amazonas|
|14|Roraima|
|15|Pará|
|16|Amapá|
|17|Tocantins|
|21|Maranhão|
|22|Piauí|
|23|Ceará|
|24|Rio Grande do Norte|
|25|Paraíba|
|26|Pernambuco|
|27|Alagoas|
|28|Sergipe|
|29|Bahia|
|31|Minas Gerais|
|32|Espírito Santo|
|33|Rio de Janeiro|
|35|São Paulo|
|41|Paraná|
|42|Santa Catarina|
|43|Rio Grande do Sul|
|50|Mato Grosso do Sul|
|51|Mato Grosso|
|52|Goiás|
|53|Distrito Federal|

> ### Sexo	
> ***

|Código|Descrição|
|---|---|
|0|Masculino|
|1|Feminino|

> ### Anos de Estudo
> ***

|Código|Descrição|
|---|---|
|1|Sem instrução e menos de 1 ano|
|2|1 ano|
|3|2 anos|
|4|3 anos|
|5|4 anos|
|6|5 anos|
|7|6 anos|
|8|7 anos|
|9|8 anos|
|10|9 anos|
|11|10 anos|
|12|11 anos|
|13|12 anos|
|14|13 anos|
|15|14 anos|
|16|15 anos ou mais|
|17|Não determinados| 
||Não aplicável|

> ### Cor
> ***

|Código|Descrição|
|---|---|
|0|Indígena|
|2|Branca|
|4|Preta|
|6|Amarela|
|8|Parda|
|9|Sem declaração|

#### Observação:
***
> Os seguintes tratamentos foram realizados nos dados originais:
> 1. Foram eliminados os registros onde a <b>Renda</b> era inválida (999 999 999 999);
> 2. Foram eliminados os registros onde a <b>Renda</b> era missing;
> 3. Foram considerados somente os registros das <b>Pessoas de Referência</b> de cada domicílio (responsável pelo domicílio).

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import binom
from scipy.stats import norm

In [2]:
# Importando a base de dados:

dados = pd.read_csv('data/dados.csv')
dados.head()

,UF,Sexo,Idade,Cor,Anos de Estudo,Renda,Altura
0,11,0,23,8,12,800,1.603808
1,11,1,23,2,12,1150,1.739790
2,11,1,35,8,15,880,1.760444
3,11,0,46,2,6,3500,1.783158
4,11,1,47,8,9,150,1.690631


***

#### Avaliando nosso dataset é possível verificar que a **proporção de homens** como chefes de domicílios é de quase **70%**. Precisamos **selecionar aleatoriamente grupos de 10 indivíduos** para verificar as diferenças entre os rendimentos em cada grupo. 

Qual a **probabilidade de selecionamos um grupo que apresente a mesma proporção da população**, ou seja, selecionarmos um grupo que seja **composto por 7 homens e 3 mulheres**?

***

In [3]:
# Verificando a real proporção entre homens e mulheres, a título de informação:

dados['Sexo'].value_counts(normalize = True)*100

0    69.299844
1    30.700156
Name: Sexo, dtype: float64

In [4]:
# Probabilidade de sortear um homem para um grupo de 10 pessoas:

p = 0.7

# probabilidade de fracasso:

q = 1 - p

In [5]:
# Número de experimentos que preciso para um grupo de 10 pessoas:

n = 10

In [6]:
# Número de eventos desejados (escolha de um homem):

k = 7

In [7]:
# Cálculo da probabilidade de, ao sortear-se aleatoriamente 10 pessoas, 7 sejam homens e 3 sejam mulheres:

# Nota: Este é um experimento binomial, ou seja, há somente 2 possibilidades: sucesso em conseguir o grupo especificado ou 
# fracasso.

prob_7 = binom.pmf(k, n, p)
print(f'A probabilidade de sortear-se 7 homens e 3 mulheres para um grupo de 10 pessoas é de {prob_7.round(2)*100}%')

A probabilidade de sortear-se 7 homens e 3 mulheres para um grupo de 10 pessoas é de 27.0%


***

#### Ainda sobre a questão anterior, **quantos grupos de 10 indivíduos** nós precisaríamos selecionar, de forma aleatória, para conseguir **100 grupos compostos por 7 homens e 3 mulheres**?

***

In [8]:
# Quantos grupos serão necessários?

media = 100

In [9]:
# Cálculo da quantidade média de grupos a serem sorteados a fim de que o experimento tenha sucesso:

# Nota: A fórmula utilizada foi a da média da distribuição binomial (média = n * p)

n = media/prob_7
n

374.7733576858062

***


Um cliente nos encomendou um estudo para avaliar o **rendimento dos chefes de domicílio no Brasil**. Para isso precisamos realizar uma nova coleta de dados, isto é, uma nova pesquisa de campo. Após reunião com o cliente foi possível elencar o seguinte conjunto de informações:

> A. O resultado da pesquisa precisa estar pronto em **2 meses**;

> B. Teremos somente **R$\$$ 150.000,00** de recursos para realização da pesquisa de campo; e
    
> C. Seria interessante uma **margem de erro não superior a 10% em relação a média estimada**.

Em nossa experiência com estudos deste tipo, sabemos que o **custo médio por indivíduo entrevistado fica em torno de R$\$$ 100,00**. Com este conjunto de fatos avalie e obtenha o seguinte conjunto de informações para passar ao cliente:


> 1. Para obter uma estimativa para os parâmetros da população (renda dos chefes de domicílio no Brasil), realize uma amostragem aleatória simples em nosso conjunto de dados. Essa amostra deve conter 200 elementos (utilize random_state = 101 para garantir que o mesmo experimento posso ser realizado novamente). Obtenha a média e o desvio-padrão dessa amostra.
    
> 2. Para a **margem de erro** especificada pelo cliente obtenha os **tamanhos de amostra** necessários para garantir os **níveis de confiança de 90%, 95% e 99%**.
    
> 3. Obtenha o **custo da pesquisa** para os três níveis de confiança.
    
> 4. Para o maior nível de confiança viável (dentro do orçamento disponível), obtenha um **intervalo de confiança para a média da população**.
    
> 5. Assumindo o **nível de confiança escolhido no item anterior**, qual **margem de erro** pode ser considerada utilizando todo o recurso disponibilizado pelo cliente?
    
> 6. Assumindo um **nível de confiança de 95%**, **quanto a pesquisa custaria ao cliente** caso fosse considerada uma **margem de erro de apenas 5%** em relação a média estimada?



***

##### Para obter uma estimativa para os parâmetros da população (renda dos chefes de domicílio no Brasil), realize uma amostragem aleatória simples em nosso conjunto de dados. Essa amostra deve conter 200 elementos (utilize random_state = 101 para garantir que o mesmo experimento posso ser realizado novamente). Obtenha a média e o desvio-padrão dessa amostra.

In [10]:
# Coletando uma amostra da renda de 200 chefes de família:

amostra_200 = dados.sample(200, random_state = 101)['Renda']

In [11]:
# Obtendo a média da amostra (média amostral): 

media_amostra = amostra_200.mean()
print(f'A renda média numa amostra aleatória composta por 200 chefes de família é de R${media_amostra.round(2)} reais')

A renda média numa amostra aleatória composta por 200 chefes de família é de R$1964.2 reais


In [12]:
# Obtendo o desvio padrão amostral:

desvio_x = amostra_200.std()
desvio_x

3139.8855167452093

##### Para a margem de erro especificada pelo cliente, obtenha os tamanhos de amostra necessários para garantir os níveis de confiança de 90%, 95% e 99%.

In [13]:
# Note que a informação sobre o erro está em percentual, isto é, um percentual relacionado à média amostral:

e = 0.10 * media_amostra
e

196.4205

In [14]:
# Calculando o tamanho da amostra para o nível de confiança igual a 90%: 

# Obtendo o valor de z:

z1 = norm.ppf(0.5 + (0.90/2))
z1

1.6448536269514722

In [15]:
# Obtendo o tamanho da amostra:

n1 = (z1 * (desvio_x/e)) ** 2
print(f'O número de entrevistas a serem realizadas a fim de se obter um intervalo de confiança de 90% é de {n1.round(0)}')

O número de entrevistas a serem realizadas a fim de se obter um intervalo de confiança de 90% é de 691.0


In [16]:
# Calculando o tamanho da amostra para o nível de confiança igual a 95%: 

# Obtendo o valor de z:

z2 = norm.ppf(0.5 + (0.95/2))
z2

1.959963984540054

In [17]:
# Obtendo o tamanho da amostra:

n2 = (z2 * (desvio_x/e)) **2
print(f'O número de entrevistas a serem realizadas a fim de se obter um intervalo de confiança de 95% é de {n2.round(0)}')

O número de entrevistas a serem realizadas a fim de se obter um intervalo de confiança de 95% é de 982.0


In [18]:
# Calculando o tamanho da amostra para o nível de confiança igual a 99%: 

# Obtendo o valor de z:

z3 = norm.ppf(0.5 + (0.99/2))
z3

2.5758293035489004

In [19]:
# Obtendo o tamanho da amostra:

n3 = (z3 * (desvio_x/e)) **2
print(f'O número de entrevistas a serem realizadas a fim de se obter um intervalo de confiança de 99% é de {n3.round(0)}')

O número de entrevistas a serem realizadas a fim de se obter um intervalo de confiança de 99% é de 1695.0


##### Obtenha o custo da pesquisa para os três níveis de confiança.


In [20]:
# Sabemos que o custo, em reais, por entrevista é de R$100,00
# Para um intervalo de confiança de 90%:

custo1 = n1 * 100
print(f'Para a quantidade de entevistas necessárias para um intervalo de confiança de 90%, isto é, {n1.round(0)} entrevistas, o valor das despesas será de {custo1.round(2)} reais')

Para a quantidade de entevistas necessárias para um intervalo de confiança de 90%, isto é, 691.0 entrevistas, o valor das despesas será de 69136.68 reais


In [21]:
# Para 95%:

custo2 = n2 * 100
print(f'Para a quantidade de entevistas necessárias para um intervalo de confiança de 95%, isto é, {n2.round(0)} entrevistas, o valor das despesas será de {custo2.round(2)} reais')

Para a quantidade de entevistas necessárias para um intervalo de confiança de 95%, isto é, 982.0 entrevistas, o valor das despesas será de 98163.53 reais


In [22]:
# Para 99%:

custo3 = n3 * 100
print(f'Para a quantidade de entevistas necessárias para um intervalo de confiança de 99%, isto é, {n3.round(0)} entrevistas, o valor das despesas será de {custo3.round(2)} reais')

Para a quantidade de entevistas necessárias para um intervalo de confiança de 99%, isto é, 1695.0 entrevistas, o valor das despesas será de 169546.24 reais


##### Para o nível de confiança de 99%, o custo da pesquisa está fora do orçamento.

##### Para o maior nível de confiança viável (dentro do orçamento disponível), obtenha um intervalo de confiança para a média da população.

In [23]:
# Obtendo o intervalo de confiança (95%) para a média da população:

intervalo = norm.interval(alpha = 0.95, loc = media_amostra, scale = desvio_x / np.sqrt(n2))
intervalo

(1767.7845, 2160.6255)

##### Assumindo o nível de confiança escolhido no item anterior, qual margem de erro pode ser considerada utilizando todo o recurso disponibilizado pelo cliente?

In [24]:
# Para a margem de erro de 95%, o experimento foi avaliado em 98.163 reais. Ou seja, sobra dinheiro.
# Com mais dinheiro, é possível aumentar o tamanho da amostra e diminuir o erro. 
# Qual será esse erro?

novo_n = 150000/100
print(f'Com o orçamento de R$150 mil reais, podem-se realizar {novo_n} entrevistas')

Com o orçamento de R$150 mil reais, podem-se realizar 1500.0 entrevistas


In [25]:
# Cálculo do novo erro para 95%:

novo_e = z2 * (desvio_x/np.sqrt(novo_n))
novo_e

158.89721122673703

In [26]:
e_percentual = novo_e/media_amostra
print(f'O novo erro estipulado, com o aumento do número de entrevistas, será de {(e_percentual*100).round(0)}%')

O novo erro estipulado, com o aumento do número de entrevistas, será de 8.0%


##### Assumindo um nível de confiança de 95%, quanto a pesquisa custaria ao cliente caso fosse considerada uma margem de erro de apenas 5% em relação a média estimada?

In [27]:
e = 0.05 * media_amostra
e

98.21025

In [28]:
z = norm.ppf(.975)
n_confianca_95 = (z * (desvio_x / e)) ** 2
n_confianca_95 = int(n_confianca_95.round())
print(f'Para um nível de confiança de 95%, devemos selecionar uma amostra de {n_confianca_95} elementos.')

Para um nível de confiança de 95%, devemos selecionar uma amostra de 3927 elementos.


In [29]:
custo_confianca_95 = n_confianca_95 * 100
print(f'Para um nível de confiança de 95% o custo da pesquisa seria de R$ {custo_confianca_95}.')

Para um nível de confiança de 95% o custo da pesquisa seria de R$ 392700.


### Testes PARAMÉTRICOS:

***

### Problema:

#### Em nosso dataset temos os rendimento dos chefes de domicílio obtidos da Pesquisa Nacional por Amostra de Domicílios - PNAD no ano de 2015. Um problema bastante conhecido em nosso país diz respeito a desigualdade de renda, principalmente entre homens e mulheres.

#### Duas amostras aleatórias, uma de **500 homens** e outra com **500 mulheres**, foram selecionadas em nosso dataset. Com o objetivo de comprovar tal desigualdade, **teste a igualdade das médias** entre estas duas amostras com um nível de **significância de 1%**.

***

In [37]:
# Selecionando as amostras:

homens = dados.query('Sexo == 0').sample(n = 500, random_state = 101)['Renda']
mulheres = dados.query('Sexo == 1').sample(n = 500, random_state = 101)['Renda']

In [34]:
# Nível de significância e de confiança:

significancia = 0.01
confianca = 1 - significancia

### Hipóteses:

* H0 : Renda média dos homens é menor ou igual à renda média das mulheres
* H1: Renda média dos homens é maior que a renda média das mulheres

##### Pretende-se mostrar que H0 não corresponde à realidade, ou seja, que a renda média dos chefes de família do sexo feminino é menor que a renda média dos chefes de família do sexo masculino.

##### Trata-se de um teste unicaudal superior.

In [33]:
# Realização do teste:

from statsmodels.stats.weightstats import DescrStatsW

teste_homens = DescrStatsW(homens)
teste_mulheres = DescrStatsW(mulheres)

In [36]:
# Comparação:

teste = teste_homens.get_compare(teste_mulheres)

In [39]:
# Aplicando o teste:

z, p_valor = teste.ztest_ind(alternative = 'larger', value = 0)
print(z)
print(p_valor)

5.865620057764754
2.2372867859458255e-09


##### Pela regra de rejeição da hipótese H0, caso o p_valor seja menor que a significância considerada, deve-se rejeitar H0. Vejamos:

In [40]:
# Comparação do p_valor com a significancia requerida:

p_valor <= significancia

True

#### Com um nível de confiança de 99%, rejeita-se a hipótese H0 e aceita-se a hipótese H1, isto é, a renda média dos homens é maior que a renda média das mulheres chefes de família, confirmando a alegação de desigualdade de renda entre os sexos.

***

### Problema: 

#### Você é um pesquisador que estuda o mercado de trabalho brasileiro e resolve estudar as diferenças salariais dos trabalhadores dos estados do Rio de Janeiro e de São Paulo. Durante sua pesquisa você verifica que, aparentemente, os rendimentos dos trabalhadores no estado do Rio de Janeiro são mais baixos que os rendimentos dos trabalhadores no estado de São Paulo. Para confirmar esta conclusão realize um teste de hipótese de comparação de médias em cima de duas amostras de trabalhadores dos dois estados. Siga as seguintes etapas:

- Selecione duas amostras de **500 trabalhadores** para cada um dos dois estados (variável UF) para realizar o teste. Utilize o **parâmetro random_state=101**.
- Considere o **nível de significância de 5%**.
- Teste a hipótese de que a **renda média dos trabalhadores do Rio de Janeiro é menor que a renda média dos trabalhadores de São Paulo**.

***

In [41]:
# Seleção das amostras:

trabalhadores_rj = dados.query('UF == 33').sample(n = 500, random_state = 101)['Renda']
trabalhadores_sp = dados.query('UF == 35').sample(n = 500, random_state = 101)['Renda']

In [42]:
# Nível de significancia e de confiança:

significancia = 0.05
confianca = 1 - significancia

### Hipóteses:

* H0: Renda média dos trabalhadores do Rio de Janeiro é maior ou igual à renda média dos trabalhadores de São Paulo
* H1: Renda média dos trabalhadores do Rio de Janeiro é menor que a renda média dos trabalhadores de São Paulo

#### Este é um teste unicaudal inferior.


In [43]:
# Realização do teste:

from statsmodels.stats.weightstats import DescrStatsW

teste_rj = DescrStatsW(trabalhadores_rj)
teste_sp = DescrStatsW(trabalhadores_sp)

In [45]:
# Comparação:

teste2 = teste_rj.get_compare(teste_sp)

In [49]:
# Obtenção do z e p_valor:

z2, p_valor2 = teste2.ztest_ind(alternative = 'smaller', value = 0)
print(z2)
print(p_valor2)

-2.255318273608558
0.012056679215693396


In [50]:
# Comparação entre o p_valor e a significância considerada no problema:

p_valor2 <= significancia

True

#### O critério de rejeição de H0 foi satisfeito, ou seja, rejeita-se, com um nível de confiança de 95%, que a renda média dos trabalhadores do RJ seja maior ou igual à renda média dos trabalhadores de SP. Pode-se, assim, afirmar que os trabalhadores de SP possuem renda maior.